In [19]:
import pandas as pd
import numpy as np 
import glob 
import warnings 
from collections import Counter
warnings.filterwarnings("ignore")
import plotly.express as px 
import seaborn as sns 
import matplotlib.pyplot as plt 
import lightgbm as lgbm 
from sklearn.model_selection import StratifiedKFold, train_test_split
import math
import os 
import random
import torch 
import pyarrow as pa
import pyarrow.parquet as pq
import torch.nn as nn
#from transformers import AdamW
from torch.utils.data import Dataset , DataLoader
from colorama import Fore , Style
r__=Fore.RED
g__=Fore.GREEN
st__=Style.RESET_ALL

In [20]:
train = pd.read_csv("optiver_raw_data/train.csv")
# get a look at train data.
train.head() 

,stock_id,time_id,target
0,0,5,0.004136
1,0,11,0.001445
2,0,16,0.002168
3,0,31,0.002195
4,0,62,0.001747


In [21]:
#Split training data into 80% train + 20% test for stocks 0-5

train = pd.read_csv("optiver_raw_data/train.csv")

time_id_of_first_80_percent = train.iloc[18384,1]


# get a look at train data for the stocks 0-5.
train = train[train["stock_id"].isin([0,1,2,3,4,5])]
train


,stock_id,time_id,target
0,0,5,0.004136
1,0,11,0.001445
2,0,16,0.002168
3,0,31,0.002195
4,0,62,0.001747
...,...,...,...
22975,5,32751,0.003966
22976,5,32753,0.001542
22977,5,32758,0.003377
22978,5,32763,0.006443


In [22]:
stock_0 = train[train["stock_id"]==0]
df_train = pd.DataFrame()
df_test = pd.DataFrame()
for i in range(6):
    stock_i = train[train["stock_id"]==i]
    split_list_i = train_test_split(stock_i, train_size = 0.8, test_size=0.2, random_state=1) 
    df_train = pd.concat([df_train, split_list_i[0]])
    df_test = pd.concat([df_test, split_list_i[1]])

In [23]:
df_train.to_csv('target_train_0to5.csv')
df_test.to_csv('target_test_0to5.csv')

In [24]:
#Split training data into 80% train + 20% test for stocks 0-5
for i in range(127):
    filename = "optiver_raw_data/book_train.parquet/stock_id=" + str(i)
    if not os.path.exists(filename):
        continue
    book_current_stock = pd.read_parquet(filename)

    stock_0_train = book_current_stock[book_current_stock['time_id'] <= time_id_of_first_80_percent]
    stock_0_train.to_parquet('stock_train/stock_' + str(i) + '_train.parquet')

    stock_0_test = book_current_stock[book_current_stock['time_id'] > time_id_of_first_80_percent]
    stock_0_test.to_parquet('stock_test/stock_' + str(i) + '_test.parquet')

